In [16]:
!pip install selenium webdriver-manager

In [62]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [63]:
# ============================
#  CLASE REUTILIZABLE (POM)
# ============================

class BasePage:

    def __init__(self, driver):
        self.driver = driver

    def click(self, locator):
        self.driver.find_element(*locator).click()

    def write(self, locator, text):
        self.driver.find_element(*locator).send_keys(text)

    def get_text(self, locator):
        return self.driver.find_element(*locator).text

    def open(self, url):
        self.driver.get(url)
        time.sleep(1)


# ============================
#  REGISTER PAGE
# ============================

class RegisterPage(BasePage):

    FIRSTNAME = (By.ID, "input-firstname")
    LASTNAME = (By.ID, "input-lastname")
    EMAIL = (By.ID, "input-email")
    PHONE = (By.ID, "input-telephone")
    PASSWORD = (By.ID, "input-password")
    CONFIRM = (By.ID, "input-confirm")
    AGREE = (By.NAME, "agree")
    CONTINUE = (By.CSS_SELECTOR, "input.btn-primary")

    def open_register(self):
        self.open("https://opencart.abstracta.us/index.php?route=account/register")

    def create_account(self, email):
        self.write(self.FIRSTNAME, "Juan")
        self.write(self.LASTNAME, "Argel")
        self.write(self.EMAIL, email)
        self.write(self.PHONE, "1234567890")
        self.write(self.PASSWORD, "Password123")
        self.write(self.CONFIRM, "Password123")
        self.click(self.AGREE)
        self.click(self.CONTINUE)
        time.sleep(2)

# ============================
#  LOGIN PAGE
# ============================

class LoginPage(BasePage):

    EMAIL = (By.ID, "input-email")
    PASSWORD = (By.ID, "input-password")
    LOGIN_BTN = (By.CSS_SELECTOR, "input.btn-primary")

    def open_login(self):
        self.open("https://opencart.abstracta.us/index.php?route=account/login")

    def login(self, email, password):
        self.write(self.EMAIL, email)
        self.write(self.PASSWORD, password)
        self.click(self.LOGIN_BTN)
        time.sleep(2)

# ============================
#  HOME PAGE
# ============================

class HomePage(BasePage):

    PRODUCT = (By.CSS_SELECTOR, ".product-layout .caption a")

    def select_first_product(self):
        product = self.driver.find_element(*self.PRODUCT)
        name = product.text
        product.click()
        time.sleep(2)
        return name

# ============================
#  MENU PAGE
# ============================        

class MenuPage(BasePage):

    TABLETS_MENU = (By.XPATH, "//a[text()='Tablets']")
    SAMSUNG_TABLET = (By.XPATH, "//a[contains(text(),'Samsung Galaxy Tab 10.1')]")

    def open_tablets(self):
        self.click(self.TABLETS_MENU)
        time.sleep(2)

    def select_samsung_tablet(self):
        self.click(self.SAMSUNG_TABLET)
        time.sleep(2)
        
# ============================
#  PRODUCT PAGE
# ============================

class ProductPage(BasePage):

    ADD_TO_CART = (By.ID, "button-cart")
    SUCCESS_MSG = (By.CSS_SELECTOR, ".alert-success")

    def add_to_cart(self):
        self.click(self.ADD_TO_CART)
        time.sleep(2)

    def get_success_message(self):
        return self.get_text(self.SUCCESS_MSG)

# ============================
#  CART PAGE
# ============================

class CartPage(BasePage):

    CART = (By.ID, "cart")
    CHECKOUT = (By.CSS_SELECTOR, "p.text-right a:nth-child(2)")

    def go_to_cart(self):
        self.click(self.CART)
        time.sleep(1)
        self.click(self.CHECKOUT)
        time.sleep(2)
        
# ============================
#  CHECKOUT PAGE
# ============================        

class CheckoutPage(BasePage):

    # Campos dirección
    FIRSTNAME = (By.ID, "input-payment-firstname")
    LASTNAME = (By.ID, "input-payment-lastname")
    ADDRESS1 = (By.ID, "input-payment-address-1")
    CITY = (By.ID, "input-payment-city")
    POSTCODE = (By.ID, "input-payment-postcode")
    COUNTRY = (By.ID, "input-payment-country")
    REGION = (By.ID, "input-payment-zone")

    # Comentario
    COMMENT_BOX = (By.NAME, "comment")

    # Payment method
    PAYMENT_COD = (By.CSS_SELECTOR, "input[value='cod']")
    TERMS_CHECKBOX = (By.NAME, "agree")
    PAYMENT_METHOD_CONTINUE = (By.ID, "button-payment-method")

    # Shipping method
    SHIPPING_CONTINUE = (By.ID, "button-shipping-method")

    # Botones continuar
    BILLING_CONTINUE = (By.ID, "button-payment-address")
    DELIVERY_CONTINUE = (By.ID, "button-shipping-address")
    CONFIRM_ORDER = (By.ID, "button-confirm")

    # Mensaje final
    SUCCESS_MSG = (By.CSS_SELECTOR, "#content h1")

    # -----------------------------
    # LLENAR DIRECCIÓN
    # -----------------------------
    def fill_billing_address(self):
        self.write(self.FIRSTNAME, "Juan")
        self.write(self.LASTNAME, "Argel")
        self.write(self.ADDRESS1, "Calle 123")
        self.write(self.CITY, "Bogotá")
        self.write(self.POSTCODE, "110111")

        country = self.driver.find_element(*self.COUNTRY)
        country.send_keys("Colombia")
        time.sleep(1)

        region = self.driver.find_element(*self.REGION)
        region.send_keys("Bogotá D.C.")
        time.sleep(1)

    # -----------------------------
    # COMENTARIO EN SHIPPING METHOD
    # -----------------------------
    def add_comment(self, text):
        comment_field = self.driver.find_element(*self.COMMENT_BOX)
        comment_field.clear()
        comment_field.send_keys(text)
        time.sleep(1)

    def shipping_method(self):
        # Escribir comentario obligatorio
        self.add_comment("Por favor entregar en horario de oficina.")

        # Continuar shipping
        self.click(self.SHIPPING_CONTINUE)
        time.sleep(2)

    # -----------------------------
    # MÉTODO DE PAGO
    # -----------------------------
    def select_payment_method(self):
        self.click(self.PAYMENT_COD)
        time.sleep(1)

        try:
            self.click(self.TERMS_CHECKBOX)
        except:
            pass

        self.click(self.PAYMENT_METHOD_CONTINUE)
        time.sleep(2)

    # -----------------------------
    # FLUJO COMPLETO DEL CHECKOUT
    # -----------------------------
    def complete_checkout(self):

        # Paso 1: Billing
        self.fill_billing_address()
        self.click(self.BILLING_CONTINUE)
        time.sleep(2)

        # Paso 2: Delivery
        self.click(self.DELIVERY_CONTINUE)
        time.sleep(2)

        # Paso 3: Shipping (incluye comentario)
        self.shipping_method()

        # Paso 4: Payment
        self.select_payment_method()

        # Paso 5: Confirmar pedido
        self.click(self.CONFIRM_ORDER)
        time.sleep(4)

    def get_confirmation_message(self):
        return self.get_text(self.SUCCESS_MSG)



In [64]:
# ============================
#  DRIVER CONFIG
# ============================

def iniciar_driver():
    options = ChromeOptions()
    options.add_argument("--start-maximized")

    # Para evitar pantalla "La conexión no es privada"
    options.accept_insecure_certs = True
    options.add_argument("--ignore-certificate-errors")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [65]:
# ============================
#  FLUJO COMPLETO
# ============================

driver = iniciar_driver()

# ============================
#  INSTANCIA DE PÁGINAS
# ============================

register = RegisterPage(driver)
login = LoginPage(driver)
menu = MenuPage(driver)           
product_page = ProductPage(driver)
cart = CartPage(driver)
checkout = CheckoutPage(driver)

# ============================
#  FLUJO COMPLETO
# ============================

email_test = f"test{int(time.time())}@demo.com" #cambia cada que se ejecute el codigo el correo

# 1. Registro
register.open_register()
register.create_account(email_test)
print("✔ Registro exitoso")

# Logout para probar login
driver.get("https://opencart.abstracta.us/index.php?route=account/logout")
time.sleep(1)

# 2. Login
login.open_login()
login.login(email_test, "Password123")
print("✔ Login exitoso")

# 3. Seleccionar producto DESDE TABLETS
driver.get("https://opencart.abstracta.us")  
menu.open_tablets()                 # <-- Navegar a Tablets
menu.select_samsung_tablet()        # <-- Seleccionar la tablet

print("✔ Tablet seleccionada")

# 4. Agregar al carrito
product_page.add_to_cart()
msg = product_page.get_success_message()
assert "Success" in msg
print("✔ Producto agregado al carrito")

# 5. Ir al carrito
cart.go_to_cart()
print("✔ Checkout iniciado")

# 6. Finalizar la compra
checkout.complete_checkout()
msg = checkout.get_confirmation_message()

assert "Your order has been placed!" in msg
print("🎉✔ COMPRA FINALIZADA EXITOSAMENTE ✔🎉")

# driver.quit()


✔ Registro exitoso
✔ Login exitoso
✔ Tablet seleccionada
✔ Producto agregado al carrito
✔ Checkout iniciado
🎉✔ COMPRA FINALIZADA EXITOSAMENTE ✔🎉
